In [26]:
import requests
import pandas as pd
import os
import pathlib
import shutil
from tqdm import tqdm

In [27]:
root = pathlib.Path("open311/cities")
if not os.path.exists(root):
    os.mkdir(root)

In [29]:
def scrape_api(city):
    print(city.city, end=" ")
    df = pd.DataFrame()
    folder = root / city.city

    if city.request_type == 0:
        i = 0
        dfs = []
        while True:
            response = requests.get(city.request_url, params={"page": i})
            if response.status_code != 200:
                break
            dfs.append(pd.DataFrame(data = response.json()))
            i += 1
        df = pd.concat(dfs)
    
    elif city.request_type == 1:
        response = requests.get(city.request_url)
        df = pd.DataFrame(data = response.json())
    
    elif city.request_type == 2:
        response = requests.get(city.request_url)
        df = pd.DataFrame(data = response.json()["service_requests"])

    else: 
        return
    
    df = df[df.media_url.str.len() > 0]
    df = df.filter(items=["service_name", "media_url"])

    os.mkdir(folder)

    print("Scraping media...")
    for sr in tqdm(df.itertuples(), total=df.shape[0]):
        sn = sr.service_name.replace("/", " bzw. ")
        media_response = requests.get(sr.media_url, stream=True)
        if media_response.status_code == 200:
            if not os.path.exists(folder / sn):
                os.mkdir(folder / sn)
            filename = sr.media_url.split("/")[-1]
            with open(folder / sn / filename, "wb") as media:
                shutil.copyfileobj(media_response.raw, media)
        

In [30]:
cities = pd.read_csv("open311/cities.csv")

for city in cities.itertuples():
    scrape_api(city)

munich Scraping media...


 24%|███████████████████████████████████████████████▎                                                                                                                                                        | 87/368 [00:19<01:00,  4.62it/s]

In [ ]:
i = 0
dfs = []
url = "https://anliegen.bonn.de/georeport/v2/requests.json"
while True:
    response = requests.get(url, params={"page": i})
    if response.status_code != 200:
        break
    dfs.append(pd.DataFrame(data = response.json()))
    i += 1
df = pd.concat(dfs)
df = df[df.media_url.str.len() > 0]
df = df.filter(items=["service_name", "media_url"])
for sr in df.itertuples():
    sn = sr.service_name.replace("/", " bzw. ")
    print(sr.media_url)
    media_response = requests.get(sr.media_url, stream=True)
    if media_response.status_code == 200:
        if not os.path.exists(root / sn):
            os.mkdir(root / sn)
        filename = sr.media_url.split("/")[-1]
        with open(root / sn / filename, "wb") as media:
            shutil.copyfileobj(media_response.raw, media)
    break